In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Removing english text , duplicates and whitespaces

In [4]:
import re
from google.colab import files

def is_telugu(text):
    cleaned = re.sub(r'[^\u0C00-\u0C7F\s]', '', text)
    return cleaned.strip()

def clean_text_lines(text):
    lines = text.split('\n')
    seen = set()
    cleaned_lines = []

    for line in lines:
        telugu_line = is_telugu(line)
        if telugu_line and telugu_line not in seen:
            seen.add(telugu_line)
            cleaned_lines.append(telugu_line)

    return '\n'.join(cleaned_lines)

input_file = "/content/Telugu_text.txt"
with open(input_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

cleaned_text = clean_text_lines(raw_text)

output_file = "/content/cleaned_telugu_only.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

# Download the cleaned file
files.download(output_file)

print("Cleaning complete")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cleaning complete


# Labeling sentiments to dataset using XLM-RoBERTa


In [5]:
!pip install -q transformers torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model & tokenizer
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [7]:
# Labels
labels = ['Negative', 'Neutral', 'Positive']

def get_sentiments_batch(texts, max_length=128):
    encodings = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    encodings = {key: val.to(device) for key, val in encodings.items()}
    with torch.no_grad():
        outputs = model(**encodings)
    probs = softmax(outputs.logits.cpu().numpy(), axis=1)
    sentiments = [labels[i.argmax()] for i in probs]
    return sentiments

input_file = "/content/cleaned_telugu_only.txt"
with open(input_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Batch processing
batch_size = 64
results = []

for i in tqdm(range(0, len(lines), batch_size)):
    batch = lines[i:i + batch_size]
    try:
        sentiments = get_sentiments_batch(batch)
        results.extend(zip(batch, sentiments))
    except Exception as e:
        print(f"Error in batch {i}: {e}")
        results.extend(zip(batch, ['Error'] * len(batch)))

df = pd.DataFrame(results, columns=["Text", "Sentiment"])
output_file = "/content/telugu_sentiment_xlmroberta.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

from google.colab import files
files.download(output_file)

print("Sentiment Analysis Complete")


100%|██████████| 2293/2293 [14:27<00:00,  2.64it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sentiment Analysis Complete


#Balancing dataset

In [8]:
import pandas as pd
from sklearn.utils import shuffle

def create_balanced_dataset(csv_path, output_path='balanced_dataset2.csv'):
    # Load the dataset
    df = pd.read_csv(csv_path)

    # Get the minimum count among the classes
    sentiment_counts = df['Sentiment'].value_counts()
    print("Original Sentiment Counts:\n", sentiment_counts)

    min_count = sentiment_counts.min()

    # Sample equal number from each sentiment
    balanced_df = pd.concat([
        df[df['Sentiment'] == 'Positive'].sample(min_count, random_state=42),
        df[df['Sentiment'] == 'Negative'].sample(min_count, random_state=42),
        df[df['Sentiment'] == 'Neutral'].sample(min_count, random_state=42)
    ])

    # Shuffle the dataset
    balanced_df = shuffle(balanced_df, random_state=42)

    # Save the balanced dataset
    balanced_df.to_csv(output_path, index=False)
    print(f"Balanced dataset saved to '{output_path}' with {len(balanced_df)} records.")

create_balanced_dataset("/content/telugu_sentiment_xlmroberta.csv")


Original Sentiment Counts:
 Sentiment
Neutral     121397
Negative     14770
Positive     10557
Name: count, dtype: int64
Balanced dataset saved to 'balanced_dataset2.csv' with 31671 records.
